In [ ]:
# working on loading data to the coco format

# system
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

# pdc
import dense_correspondence_manipulation.utils.utils as utils
from  dense_correspondence_manipulation.coco.coco_dataset_formatter import COCOSpartanDatasetLoader, COCODataFormatter

%load_ext autoreload
%autoreload 2
%matplotlib inline

DATASET_NAME = "multi_shoe_val_10"
MIN_NUM_SHOES = 1
MAX_NUM_SHOES = 7
NUM_IMAGES = 10

In [ ]:
config_filename = os.path.join(utils.getDenseCorrespondenceSourceDir(), 'config', 'dense_correspondence', 
                               'dataset', 'composite', 'shoes_all.yaml')

train_config_filename = os.path.join(utils.getDenseCorrespondenceSourceDir(), 'config', 'dense_correspondence', 
                               'training', 'training.yaml')

data_loader = COCOSpartanDatasetLoader(config_filename, train_config_filename, random=True)
data_formatter = COCODataFormatter(DATASET_NAME)

In [ ]:
image_rgb, image_mask, rgb_filename = data_formatter.get_random_merged_image(data_loader, MIN_NUM_SHOES, MAX_NUM_SHOES)

# display the original RGB image and the mask
plt.figure(1,figsize=(10, 8))
plt.subplot(121)
plt.imshow(image_rgb)

plt.subplot(122)
plt.imshow(image_mask)

In [ ]:
# process of creating coco annotations

# keep track of images and annotations
images = []
annotations = []

annotation_id = 0

# grab a number of random images

image_id = 0

import time
start = time.time()


while image_id < NUM_IMAGES:
    
    start_time = time.time()
    
    
    # load a new data point
    # 1 to 7 shoes per image
    image_rgb, image_mask, rgb_filename = data_formatter.get_random_merged_image(data_loader, MIN_NUM_SHOES, MAX_NUM_SHOES)
    
    # TODO(ethan): set width, height, and file_name
    height, width = data_formatter.get_height_and_width_of_image_mask(image_mask)
    # COCO doesn't expect the full path
    file_name = os.path.basename(rgb_filename)
    
    image_info = data_formatter.get_image_info(
    image_id,
    width,
    height,
    file_name)
    
    images.append(image_info)
    
    # iterate through all the masks in the image (in this case assuming one)
    number_masks = np.max(image_mask)
    for object_id in range(1):
        # object_id (our notation) is category_id (COCO notation)
        category_id = object_id
        
        # iterate over each unique shoe and create an annotation
        for mask_idx in range(1, number_masks+1):
            try:
                # create a binary mask of just the unique shoe
                this_mask = image_mask.copy()
                this_mask[image_mask != mask_idx] = 0
                this_mask[image_mask == mask_idx] = 1
                annotation_info = data_formatter.get_annotation_info(annotation_id, image_id, category_id, this_mask)
            except:
                # the area of the mask could be 0
                continue
            annotations.append(annotation_info)
            annotation_id += 1
        
    image_id += 1
    
    elapsed = time.time() - start_time
    print("image %d of %d, duration %.2f" %(image_id, NUM_IMAGES, elapsed))
    
done = time.time()
elapsed = done - start
print("Time to complete: {} seconds".format(elapsed))

coco_output = data_formatter.get_coco_output_from_images_and_annotations(images, annotations)
data_formatter.write_to_relative_json_filename(coco_output)